In [23]:
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


## 环境变量

In [24]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
# print(os.environ)

AI_TEMPERATURE=0.0
AI_MAX_TOKENS=2048

## 模型加载

In [25]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import AzureOpenAI
from langchain.chat_models import AzureChatOpenAI

def load_azureLLM():
    # chat
    chat = AzureChatOpenAI(
        deployment_name=os.environ.get('DEPLOYMENT_NAME_CHAT'),
        temperature=AI_TEMPERATURE,
        max_tokens=AI_MAX_TOKENS,
        )
    # llm
    llm = AzureOpenAI(
        deployment_name=os.environ.get('DEPLOYMENT_NAME_LLM'),
        model_name="text-davinci-003",
        temperature=AI_TEMPERATURE,
        max_tokens=AI_MAX_TOKENS,
        )
    
    embedding = OpenAIEmbeddings(deployment = os.environ.get('DEPLOYMENT_NAME_EMBEDDING'),chunk_size=1)
    
    return llm,chat,embedding

llm,chat,embed = load_azureLLM()

In [26]:
from langchain.agents import load_tools
from langchain.agents import AgentType
from langchain.agents import initialize_agent

In [27]:
tools = load_tools(llm=chat,
                   tool_names=["llm-math","serpapi"],verbose=True)

In [28]:
agent = initialize_agent(llm=llm,
                         tools=tools,
                         agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                         verbose=True)

In [29]:
agent.run("美国现任总统是谁？他的夫人多少岁？")

Error in on_chain_start callback: 'name'


 I need to find out who the current president of the US is and how old his wife is.
Action: Search
Action Input: "current president of the US"
Observation: Joe Biden
Thought: I need to find out how old his wife is.
Action: Search
Action Input: "age of Jill Biden"
Observation: 72 years
Thought: I now know the final answer.
Final Answer: Joe Biden is the current president of the US and his wife, Jill Biden, is 72 years old.

> Finished chain.


'Joe Biden is the current president of the US and his wife, Jill Biden, is 72 years old.'

In [33]:
from langchain.agents import Tool
from langchain.agents import initialize_agent
from langchain.agents import AgentType

from langchain import Wikipedia

from langchain.agents.react.base import DocstoreExplorer

docstore = DocstoreExplorer(Wikipedia())
tools = [
    Tool(
        name="Search",
        func=docstore.search,
        description="在需要搜索时非常有用",
    ),
    Tool(
        name="Lookup",
        func=docstore.lookup,
        description="在需要查询时非常有用",
    ),
]

In [34]:
react = initialize_agent(llm=llm,
                         tools=tools,
                         agent=AgentType.REACT_DOCSTORE,
                         verbose=True)

In [36]:
react.run("陆奇当前的工作重点是什么？")

Error in on_chain_start callback: 'name'


Thought: 我需要搜索陆奇，找到他当前的工作重点。
Action: Search[陆奇]
Observation: Lu Qi (simplified Chinese: 陆奇; traditional Chinese: 陸奇; pinyin: Lù Qí; born September 3, 1961) is a Chinese-American software executive and engineer who is the head of MiraclePlus, a startup incubator in China. Previously, Lu was the head of Y Combinator China until it was shut down. He was formerly chief operating officer of Baidu until he stepped down in May 2018. He has served as executive vice president of Microsoft, leading development of Bing, Skype, and Microsoft Office, and software engineer and manager for Yahoo!'s search technology division.
Thought: 陆奇当前的工作重点是创办MiraclePlus创业孵化器。
Action: Finish[创办MiraclePlus创业孵化器]

> Finished chain.


'创办MiraclePlus创业孵化器'